In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [3]:
# import data set
train = pd.read_csv('data/naive_bayes_train.csv')
test = pd.read_csv('data/naive_bayes_train.csv')

In [4]:
train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [9]:
# insincere has target 1
# sincere has target 0
train[train.target == 1].sample(5)

,qid,question_text,target
1231273,f149df66b98147046031,How it is possible to sex with mom?,1
1134748,de685e8057015762908d,When will academia stop perpetuating the lie t...,1
1061175,cfee1f3c369dde467e41,Does hindutva ask to kill human in the name of...,1
24585,04d1b9adfe8071da6e51,"Do Trump haters, by definition, realize that t...",1
1055404,ced0c56b188f5cdc79c5,Do Brahmins have Iranian ancestry? Why aren't ...,1


In [10]:
train[train.target == 0].sample(5)

,qid,question_text,target
1217012,ee854767bc32394a4e3d,Why is Wendy Williams proportional with to a s...,0
675801,845965ff38cf622a041f,Why do I get so many calls where the caller ju...,0
851469,a6d53e1aa25db5195c1e,What are the pros and cons of commercial armies?,0
597678,7511968f236906ea76b3,How is sadness and pleasure similar?,0
315856,3de7d29bed4c9c1ef1df,What are the must visit points in VASCO and mu...,0


In [12]:
samp = train.sample(1)
sentence = samp.iloc[0]['question_text']
print (sentence)

Can we build up a strong love affair with a person who is 9 years older than us?


In [13]:
# preprocess text
import re
sentence = re.sub(r'\d+', '', sentence)
print('Sentence after removing numbers\n', sentence)
#Removing Punctuations in a string.

import string
sentence = sentence.translate(sentence.maketrans("","",string.punctuation))
print ('Sentence After Removing Punctuations\n',sentence)

Sentence after removing numbers
 Can we build up a strong love affair with a person who is  years older than us?
Sentence After Removing Punctuations
 Can we build up a strong love affair with a person who is  years older than us


In [14]:
# use nltk to remove stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))
words_in_sentence = list(set(sentence.split(' ')) - stop_words)
print (words_in_sentence)

['', 'strong', 'Can', 'build', 'years', 'affair', 'older', 'us', 'love', 'person']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# stemmer from nltk
from nltk.stem import PorterStemmer
nltk.download('wordnet')
stemmer= PorterStemmer()
for i,word in enumerate(words_in_sentence):
    words_in_sentence[i] = stemmer.stem(word)
print (words_in_sentence)   

['', 'strong', 'can', 'build', 'year', 'affair', 'older', 'us', 'love', 'person']


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jared\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size=0.2)

In [17]:
word_count = {}
word_count_sincere = {}
word_count_insincere = {}
sincere  = 0
insincere = 0 

In [18]:
stemmer = PorterStemmer()

In [20]:
row_count = train.shape[0]
for row in range(0,row_count):
    insincere += train.iloc[row]['target']
    sincere += (1 - train.iloc[row]['target'])
    sentence = train.iloc[row]['question_text']
    sentence = re.sub(r'\d+','',sentence)
    sentence = sentence.translate(sentence.maketrans("","",string.punctuation))
    words_in_sentence = list(set(sentence.split(' ')) - stop_words)
    for index,word in enumerate(words_in_sentence):
        word = stemmer.stem(word)
    for word in words_in_sentence:
        if train.iloc[row]['target'] == 0:   #Sincere Words
            if word in word_count_sincere.keys():
                word_count_sincere[word]+=1
            else:
                word_count_sincere[word] = 1
        elif train.iloc[row]['target'] == 1: #Insincere Words
            if word in word_count_insincere.keys():
                word_count_insincere[word]+=1
            else:
                word_count_insincere[word] = 1
        if word in word_count.keys():        #For all words. I use this to compute probability.
            word_count[word]+=1
        else:
            word_count[word]=1

print('Done')

In [ ]:
word_probability = {}
total_words = 0
for i in word_count:
    total_words += word_count[i]
for i in word_count:
    word_probability[i] = word_count[i] / total_words

#Eliminating words which are insignificant. Insignificant words are words which have a probability of occurence less than 0.0001.
print ('Total words ',len(word_probability))
print ('Minimum probability ',min (word_probability.values()))
threshold_p = 0.0001
for i in list(word_probability):
    if word_probability[i] < threshold_p:
        del word_probability[i]
        if i in list(word_count_sincere):   #list(dict) return it;s key elements
            del word_count_sincere[i]
        if i in list(word_count_insincere):  
            del word_count_insincere[i]
print ('Total words ',len(word_probability))